# ch07-종합을 위한 그룹화, 필터링, 그리고 변환

- 데이터 분석 중 가장 기본적인 과제는 각 그룹에 대한 계산을 수행하기 전에 데이터를 독립적인 그룹으로 분할하는 것이다.
- 분할-적용-병합 split-apply-combine 으로 불린다.
- groupby() 메서드의 결과는 groupby 객체다.
- groupby 객체가 생성될 때 pandas가 하는 일을 별로 없고 단지 주어진 조건의 그룹화가 가능한지 확인만 한다.
- 제대로 사용하려면 groupby 객체에 메서드를 체인시켜야 한다.

In [2]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series

## 1.종합 aggregation에 대한 정의

- groupby() 메서드의 가장 보편적인 사용법은 종합 aggregation이다.
- 데이터 분석에서 종합이란 여러 입력이 합산되거나 병합되어 단일 출력값으로 만들어지는 것을 의미한다.
- col의 모든 값을 더하거나 최대값을 찾는 것이 연속된 단일 데이터에 종합을 적용한 예다.
- 주요한 종합 함수로는 sum, min, max, mean, count, variance, std 등이 있다.

- 비행 데이터셋을 조사하고 가장 간단한 형태일 단일 그룹화 열, 단일 열 종합, 단일 종합 함수만 사용하는 종합을 수행해본다.
- 각 항공사의 평균 연착율을 알아보자.

- 항공 데이터셋을 읽어들인다.

In [3]:
flights = pd.read_csv('../data/flights.csv')

In [5]:
flights.head()

,MONTH,DAY,WEEKDAY,AIRLINE,ORG_AIR,DEST_AIR,SCHED_DEP,DEP_DELAY,AIR_TIME,DIST,SCHED_ARR,ARR_DELAY,DIVERTED,CANCELLED
0,1,1,4,WN,LAX,SLC,1625,58.0,94.0,590,1905,65.0,0,0
1,1,1,4,UA,DEN,IAD,823,7.0,154.0,1452,1333,-13.0,0,0
2,1,1,4,MQ,DFW,VPS,1305,36.0,85.0,641,1453,35.0,0,0
3,1,1,4,AA,DFW,DCA,1555,7.0,126.0,1192,1935,-7.0,0,0
4,1,1,4,WN,LAX,MCI,1720,48.0,166.0,1363,2225,39.0,0,0


- groupby() 메서드에 그룹화할 col을 전달하고 agg 메서드에 종합할 col과 종합 함수를 딕셔너리 형태로 전달하여 호출한다.

In [6]:
flights.groupby('AIRLINE').agg({'ARR_DELAY':'mean'}).head()

,ARR_DELAY
AIRLINE,
AA,5.542661
AS,-0.833333
B6,8.692593
DL,0.339691
EV,7.034580


- 또 다른 방법으로는 종합할 col을 index operator에 놓고 종합 함수를 agg의 문자열로 전달할 수도 있다.

In [9]:
flights.groupby('AIRLINE')['ARR_DELAY'].agg('mean').head()

AIRLINE
AA    5.542661
AS   -0.833333
B6    8.692593
DL    0.339691
EV    7.034580
Name: ARR_DELAY, dtype: float64

- 함수 이름을 문자열로 사용할 수 있는 것은 특정 종합 함수를 문자열로 참고할 수 있도록 pandas에 구현되어 있는 기능이다.
- numpy의 mean 함수를 사용한 방법처럼 어떤 종합 함수라도 직접 agg 메서드에 전달할 수 있다.

In [10]:
flights.groupby('AIRLINE')['ARR_DELAY'].agg(np.mean).head()

AIRLINE
AA    5.542661
AS   -0.833333
B6    8.692593
DL    0.339691
EV    7.034580
Name: ARR_DELAY, dtype: float64

- 이 예제의 경우 agg() 메서드를 쓰지 않고 바로 mean() 메서드를 사용할 수도 있다.

In [11]:
flights.groupby('AIRLINE')['ARR_DELAY'].mean().head()

AIRLINE
AA    5.542661
AS   -0.833333
B6    8.692593
DL    0.339691
EV    7.034580
Name: ARR_DELAY, dtype: float64

- groupby() 메서드의 결과에 메서드 체인을 하는 대신 그 결과 자체를 변수에 저장해보자.

In [12]:
grouped = flights.groupby('AIRLINE')

In [13]:
type(grouped)

pandas.core.groupby.DataFrameGroupBy

- 이 단계에서는 어떤한 계산도 일어나지 않는다.
- pandas는 단지 그룹화된 열을 검증하지만 한다.
- groupby 객체는 aggregation 연산을 수행하기 위한 agg() 메서드를 갖고 있다.
- 이 메서드를 사용하는 방법 중 하나는 앞에서 본 것처럼 이 객체에 aggregation col을 aggregation 함수로 매핑하는 딕셔너리를 전달하는 것이다.

- 몇 가지 다른 문법을 사용할 수 있다.
- 딕셔너리 대신 DataFrame에서 col을 선택하는 것처럼 index operator를 사용할 수도 있다.
- 그 경우 함수의 문자열 이름이 agg() 메서드에 스칼라 값으로 잔달된다.

- 단일 aggregation 함수만 적용하면 agg() 메서드 없이 groupby 객체 자체의 메서드를 직접 호출할 수 있다.
- 다음에 열거된 aggregation 함수들은 agg() 메서드에 문자열로 전달하거나 groupby 객체에 메서드로 직접 사용할 수 있다.
    - min
    - max
    - mean
    - median
    - sum
    - count
    - std
    - var
    - size
    - describe
    - nunique
    - idxmin
    - idxmax

- agg() 메서드에 aggregation 함수가 아닌 것을 사용하면 ValueError가 발생한다.


## 2.복수 열과 함수를 사용한 그룹화와 종합
## 3.그룹화 후 MultiIndex 제거
## 4.종합 함수 커스터마이즈
## 5.*arg, **kwargs 함수를 사용한 종합 커스터마이즈
## 6. groupby 객체 조사
## 7.소수 인종이 다수인 주 찾기
## 8.체중 감량 내기를 통한 변환
## 9.apply를 사용해 주별 SAT 가중 평균 구하기
## 10.연속 변수에 대한 그룹화
## 11.도시 간 총 비행 횟수 계산
## 12.최장 연속 정시 운행 찾기